In [1]:
import torch
import pandas
from BiasMFRecommender import BiasMF
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
import numpy
import platform
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.callbacks import RichProgressBar

In [2]:
class RateDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __getitem__(self, index):
        x = torch.tensor(numpy.append(self.df.user[index] -  1, self.df.movie[index] - 1))
        y = torch.tensor(self.df.rating[index]).to(torch.float32)
        return x, y

    def __len__(self):
        return self.df.shape[0]
    
def get_loss(df, model):
    with torch.no_grad():
        criterion = torch.nn.MSELoss()
        preds = model(torch.tensor(df.user - 1), torch.tensor(df.movie - 1))
        return criterion(preds, torch.tensor(df.rating))

In [3]:
COLS = ['user', 'movie', 'rating', 'timestamp']
# df_train = pandas.read_csv("./data/ml-100k/u1.base", sep='\t', names=COLS).drop(columns=['timestamp']).astype(int)
# df_test = pandas.read_csv("./data/ml-100k/u1.test", sep='\t', names=COLS).drop(columns=['timestamp']).astype(int)
df_1m = pandas.read_csv("./data/ml-1m/ratings.dat", sep='::', names=COLS, engine='python').drop(columns=['timestamp']).astype(int)
df_train, df_test = train_test_split(df_1m, test_size=0.2, random_state=42, shuffle=True)
df_train = df_train.reset_index()
df_test = df_test.reset_index()
train_data = RateDataset(df_train)
test_data = RateDataset(df_test)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
validation_loader = DataLoader(test_data, batch_size=32, shuffle=True)
print(df_train.shape, df_test.shape)

(800167, 4) (200042, 4)


In [ ]:
loader_iterator = enumerate(train_loader)
first_batch = next(loader_iterator)
print(first_batch[1][0][:,0])
print(first_batch[1][0][:,1])
print(first_batch)

In [4]:
class LitModel(pl.LightningModule):
    def __init__(self, params):
        super().__init__()
        self.num_users = params['num_users']
        self.num_items = params['num_items']
        self.latent_dim = params['latent_dim']
        self.mu = params['global_mean']

        self.user_embedding = torch.nn.Embedding(self.num_users, self.latent_dim)
        self.item_embedding = torch.nn.Embedding(self.num_items, self.latent_dim)

        self.user_bias = torch.nn.Embedding(self.num_users, 1)
        self.user_bias.weight.data = torch.zeros(self.num_users, 1).float()
        self.item_bias = torch.nn.Embedding(self.num_items, 1)
        self.item_bias.weight.data = torch.zeros(self.num_items, 1).float()
        
        self.loss_fn = torch.nn.MSELoss()

    def forward(self, x):
        user_indices = x[:,0]
        item_indices = x[:,1]
        user_vec = self.user_embedding(user_indices)
        item_vec = self.item_embedding(item_indices)
        dot = torch.mul(user_vec, item_vec).sum(dim=1)
        rating = dot + self.mu + torch.squeeze(self.user_bias(user_indices)) + torch.squeeze(self.item_bias(item_indices))
        return rating

    def training_step(self, batch, batch_idx):
        X, y = batch
        pred = model(X)
        train_loss = self.loss_fn(pred, y)
        self.log('training loss', train_loss, on_step=True, on_epoch=True)
        return train_loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        pred = model(x)
        val_loss = self.loss_fn(pred, y)
        self.log('validation loss', val_loss, on_step=True, on_epoch=True)
        return val_loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0001)

In [8]:
params = {'num_users': df_1m.user.max(), 'num_items': df_1m.movie.max(), 'global_mean': df_1m.rating.mean(), 'latent_dim': 0}
print (torch.__version__, torch.backends.mps.is_available(),platform.processor()) 
trainer = pl.Trainer(max_epochs=5, callbacks=[TQDMProgressBar(refresh_rate=1000)]) 
# , accelerator='mps', devices=1)
model = LitModel(params)
trainer.fit(model, train_dataloaders=train_loader,val_dataloaders=validation_loader)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 0     
1 | item_embedding | Embedding | 0     
2 | user_bias      | Embedding | 6.0 K 
3 | item_bias      | Embedding | 4.0 K 
4 | loss_fn        | MSELoss   | 0     
---------------------------------------------
10.0 K    Trainable params
0         Non-trainable params
10.0 K    Total params
0.040     Total estimated model params size (MB)


1.13.1 True arm


Sanity Checking: 0it [00:00, ?it/s]

/Users/skelley/opt/anaconda3/envs/my_torch_arm/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/skelley/opt/anaconda3/envs/my_torch_arm/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/skelley/opt/anaconda3/envs/my_torch_arm/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider incr

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [8]:
params = {'num_users': df_train.user.max(), 'num_items': df_train.movie.max(), 'global_mean': df_train.rating.mean(), 'latent_dim': }
model = BiasMF(params)
model.load_state_dict(torch.load( "./saved_models/matrix_movielens_2.pth"))

SyntaxError: invalid syntax (3125296633.py, line 1)

In [42]:
df_history

,latent_dim,train_loss,test_loss
800167,100,tensor(0.4463),tensor(4.1441)


In [21]:
print(get_loss(df_train, model))
print(get_loss(df_test, model))

tensor(0.7937)
tensor(0.9582)


In [45]:
with torch.no_grad():
    print(model(torch.tensor([65]),torch.tensor([29])))
    print(model.user_bias.weight[65] + model.item_bias.weight[29] + model.mu)

tensor([4.1378])
tensor([4.1378])


In [44]:
model

BiasMF(
  (user_embedding): Embedding(943, 0)
  (item_embedding): Embedding(1682, 0)
  (user_bias): Embedding(943, 1)
  (item_bias): Embedding(1682, 1)
)

In [46]:
params

{'num_users': 943, 'num_items': 1682, 'global_mean': 3.52835, 'latent_dim': 0}